In [1]:
import os
os.chdir('../')

In [15]:
import mlflow
import tensorflow as tf
from src.utils.utlis import read_yaml,create_directories,save_json
from src.constants import *
import mlflow.keras
from urllib.parse import urlparse

In [ ]:
os.environ['MLFLOW_TRACKING_URI']='https://dagshub.com/AIAkashMukherjee/Kidney-Disease.mlflow'
os.environ['MLFLOW_TRACKING_USERNAME']='AIAkashMukherjee'


In [17]:
model = tf.keras.models.load_model("artifacts/training/model.h5")

In [18]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    model_path:Path
    train_data:Path
    params:dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

In [22]:

class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def eval_config(self):
        eval_config=EvaluationConfig(
            model_path="artifacts/training/model.h5",
            train_data="artifacts/data_ingestion/kidney-ct-scan-image",
            mlflow_uri=os.environ['MLFLOW_TRACKING_URI'],
            params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )    
        return eval_config

In [31]:
class Evaluation:
    def __init__(self,config:EvaluationConfig):
        self.config=config

    def train_valid_generator(self):
        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.train_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)

    def evaluation(self):
        self.model=self.load_model(self.config.model_path)
        self.train_valid_generator()
        self.score=model.evaluate(self.valid_generator)
        self.save_score()

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)   

    def log_with_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        with mlflow.start_run():
            mlflow.log_params(self.config.params)
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy": self.score[1]}
            )

            if tracking_url_type_store != "file":
                mlflow.keras.log_model(self.model, "model", registered_model_name="VGG16Model")
            else:
                mlflow.keras.log_model(self.model, "model")
         
        
    

In [32]:
try:
    config = ConfigurationManager()
    eval_config = config.eval_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_with_mlflow()

except Exception as e:
   raise e

Found 139 images belonging to 2 classes.


/Users/akashmukherjee/Programming/Practise ML/Kidney_Disease/my_env/lib/python3.11/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


9/9 ━━━━━━━━━━━━━━━━━━━━ 9s 972ms/step - accuracy: 1.0000 - loss: 0.0189


2024/11/14 09:20:12 WARNING mlflow.keras.save: You are saving a Keras model without specifying model signature.
2024/11/14 09:20:16 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Successfully registered model 'VGG16Model'.
2024/11/14 09:20:33 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: VGG16Model, version 1
Created version '1' of model 'VGG16Model'.
2024/11/14 09:20:33 INFO mlflow.tracking._tracking_service.client: 🏃 View run useful-yak-802 at: https://dagshub.com/AIAkashMukherjee/Kidney-Disease.mlflow/#/experiments/0/runs/6333993fd5a546b0b49fb349760149d9.
2024/11/14 09:20:33 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://dagshub.com/AIAkashMukherjee/Kidney-Disease.mlflow/#/experiments/0.
